# Liliput report queries 

* https://docs.google.com/document/d/1HafQvQiMa2VMcMyRe08swL6Cg47HCNOjp-g-FUwslpE/edit#heading=h.asdkblljy1p

* https://lilliput.dimensions.ai/discover/publication

## Prerequisites

Connect to the DB

### Connect using local credentials..

In [1]:
!pip install google-cloud-bigquery -U --quiet
from google.cloud import bigquery
import os
project_id = "ds-data-solutions-gbq"
os.environ["GCLOUD_PROJECT"] = project_id
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/michele.pasin/.config/gcloud/application_default_credentials.json"
client = bigquery.Client()
%load_ext google.cloud.bigquery

### Or connect using Colab auth..

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [2]:
project_id = "ds-data-solutions-gbq"
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


### Set up params variable


In [3]:
# eg for params
bq_params = {}
bq_params["journal_id"] = "jour.1115214"

## 1. basic stats

### Publications

In [3]:
%%bigquery --params $bq_params --project $project_id 

with liliput as (
  SELECT researcher_id
  from `ds-data-solutions-gbq.liliput01.researchers`
)
select count(distinct p.id) as tot, year 
  from `dimensions-ai.data_analytics.publications` p, unnest(author_researcher_ids) allres
  join liliput on liliput.researcher_id = allres
group by year 
order by year asc

tot    year
0      1     NaN
1      2  1948.0
2      6  1949.0
3     10  1950.0
4     11  1951.0
..   ...     ...
69  1088  2016.0
70   990  2017.0
71   982  2018.0
72   999  2019.0
73   445  2020.0

[74 rows x 2 columns]

### Datasets 

In [5]:
%%bigquery --params $bq_params --project $project_id 

with liliput as (
  SELECT researcher_id
  from `ds-data-solutions-gbq.liliput01.researchers`
)
select count(distinct p.id) as tot, EXTRACT(YEAR FROM CAST(publication_date AS DATE)) as year  
  from `dimensions-ai.data_analytics.data_sets` p, unnest(researcher_ids) allres
  join liliput on liliput.researcher_id = allres
group by year 
order by year asc

tot  year
0     4  1996
1     1  1998
2    28  1999
3     3  2000
4     6  2002
5     6  2003
6     2  2004
7    14  2005
8    10  2006
9    21  2007
10   13  2008
11   35  2009
12   17  2010
13   32  2011
14   28  2012
15   81  2013
16   51  2014
17   81  2015
18   79  2016
19   40  2017
20   30  2018
21   63  2019
22   33  2020

### Patents

can't be done as we don't have researchers links

In [6]:
# %%bigquery --params $bq_params --project $project_id 

# with liliput as (
#   SELECT researcher_id
#   from `ds-data-solutions-gbq.liliput01.researchers`
# )
# select count(distinct p.id) as tot, year 
#   from `dimensions-ai.data_analytics.patents` p, unnest(researcher_ids) allres
#   join liliput on liliput.researcher_id = allres
# group by year 
# order by year asc

### Grants

including active and inactive infos


In [7]:
%%bigquery --params $bq_params --project $project_id 

with liliput as (
  SELECT researcher_id
  from `ds-data-solutions-gbq.liliput01.researchers`
)
select count(distinct p.id) as tot, start_year, 
  CASE
    WHEN PARSE_DATE('%Y-%m-%d', end_date) >= CURRENT_DATE() THEN "active"
    ELSE "inactive"
  END AS status
  from `dimensions-ai.data_analytics.grants` p, unnest(researchers) allres
  join liliput on liliput.researcher_id = allres.id
group by start_year, status 
order by start_year asc

tot  start_year    status
0     5         NaN  inactive
1     1      1968.0  inactive
2     1      1969.0  inactive
3     1      1970.0  inactive
4     2      1971.0  inactive
..  ...         ...       ...
73    7      2018.0    active
74    6      2018.0  inactive
75   14      2019.0    active
76    2      2019.0  inactive
77    3      2020.0    active

[78 rows x 3 columns]

### Cl Trials

In [8]:
%%bigquery --params $bq_params --project $project_id 

with liliput as (
  SELECT researcher_id
  from `ds-data-solutions-gbq.liliput01.researchers`
)
select count(distinct p.id) as tot, start_year
  from `dimensions-ai.data_analytics.clinical_trials` p, unnest(investigators) allres
  join liliput on liliput.researcher_id = allres.id
group by start_year 
order by start_year asc

tot  start_year
0    1        2005
1    1        2009
2    1        2012
3    2        2013
4    1        2014

## 2. researchers 


In [9]:
%%bigquery --params $bq_params --project $project_id 

with liliput as (
  SELECT researcher_id
  from `ds-data-solutions-gbq.liliput01.researchers`
)
select count(distinct researcher_id) as tot
  from liliput

tot
0  138

### Demographics

In [10]:
%%bigquery --params $bq_params --project $project_id 

SELECT count(distinct researcher_id) as researchers, Gender, DATE_DIFF(CURRENT_DATE(), Date_of_Birth, YEAR) as age,
  CASE
    WHEN DATE_DIFF(CURRENT_DATE(), Date_of_Birth, YEAR) > 70 THEN "70+"
    WHEN DATE_DIFF(CURRENT_DATE(), Date_of_Birth, YEAR) > 60 THEN "60+"
    WHEN DATE_DIFF(CURRENT_DATE(), Date_of_Birth, YEAR) > 50 THEN "50+"
    WHEN DATE_DIFF(CURRENT_DATE(), Date_of_Birth, YEAR) > 40 THEN "40+"
    WHEN DATE_DIFF(CURRENT_DATE(), Date_of_Birth, YEAR) > 30 THEN "30+"
    ELSE "0-30"
  END AS age_group
  from `ds-data-solutions-gbq.liliput01.researchers`
  group by Gender, age, age_group

researchers  Gender   age age_group
0             1    Male  83.0       70+
1             1  Female  73.0       70+
2             1    Male  73.0       70+
3             2    Male  80.0       70+
4             1    Male  84.0       70+
5             1    Male  76.0       70+
6             1    Male  67.0       60+
7             1    Male  88.0       70+
8             1    Male  99.0       70+
9             1    Male  92.0       70+
10            1    Male  68.0       60+
11            1  Female  72.0       70+
12            1    Male  96.0       70+
13            1  Female  59.0       50+
14          123    None   NaN      0-30

### Collaborations including geo information

In [11]:
%%bigquery --params $bq_params --project $project_id 

with liliput as (
  SELECT researcher_id
  from `ds-data-solutions-gbq.liliput01.researchers`
),
grids_list as (
  select count(distinct p.id) as pubs, grid_id
    from `dimensions-ai.data_analytics.publications` p, unnest(author_researcher_ids) allres, unnest(author_grid_ids) grid_id
    join liliput on liliput.researcher_id = allres
    group by grid_id
) 
select pubs, grid_id, grid.name, geo.country, geo.city, CONCAT(geo.lat, ', ', geo.lng) AS latlong
from grids_list
join `dimensions-ai.data_analytics.grid` grid on grid.id = grids_list.grid_id, UNNEST(addresses) as geo
order by pubs desc


pubs        grid_id                                               name  \
0     1381  grid.168010.e                                Stanford University   
1     1153  grid.410786.c                                Kitasato University   
2     1119    grid.4830.f                            University of Groningen   
3     1076  grid.38142.3c                                 Harvard University   
4     1061  grid.258799.8                                   Kyoto University   
...    ...            ...                                                ...   
3790     1  grid.266518.e                              University of Karachi   
3791     1  grid.418953.2                 Institute of Cytology and Genetics   
3792     1    grid.6083.d  National Centre of Scientific Research Demokritos   
3793     1  grid.22459.38              National Hellenic Research Foundation   
3794     1  grid.414818.0  Fondazione IRCCS Ca' Granda Ospedale Maggiore ...   

            country         city                                  latlong  
0     United States     Stanford  37.430000305175781, -122.16999816894531  
1             Japan        Tokyo   35.642559051513672, 139.72563171386719  
2       Netherlands    Groningen   53.219444274902344, 6.5629658699035645  
3     United States    Cambridge  42.377052307128906, -71.116653442382812  
4             Japan        Kyoto      35.026157379150391, 135.77978515625  
...             ...          ...                                      ...  
3790       Pakistan      Karachi       24.9408016204834, 67.1202392578125  
3791         Russia  Novosibirsk   54.847682952880859, 83.107002258300781  
3792         Greece       Athens     37.9991455078125, 23.818248748779297  
3793         Greece       Athens   37.974067687988281, 23.745809555053711  
3794          Italy        Milan     45.45867919921875, 9.197662353515625  

[3795 rows x 6 columns]

## 3. Publishing Behaviour

### Fields of Research

NOTE comparisons are made across publications that HAVE at least one FOR code

In [15]:
%%bigquery --params $bq_params --project $project_id 

-- only pubs with an FOR
WITH liliput_pubs AS 
(
  SELECT *
  FROM `dimensions-ai.data_analytics.publications` p, UNNEST(author_researcher_ids) allres
  JOIN `ds-data-solutions-gbq.liliput01.researchers` r ON r.researcher_id = allres
  WHERE ARRAY_LENGTH(category_for) > 0    
), 
stats_global AS 
(
    
    SELECT cat.first_level.name, COUNT(distinct p.id) AS pubs_global, 
        ROUND (
            (COUNT(distinct p.id) * 100 /  -- calc percentage to total pubs - global
                    (
                    SELECT COUNT(distinct id) 
                    FROM `dimensions-ai.data_analytics.publications` 
                    WHERE ARRAY_LENGTH(category_for) > 0
                    )
                ), 
            2 )  AS pubs_global_pc 
    FROM `dimensions-ai.data_analytics.publications` p, unnest(category_for) cat
    GROUP BY cat.first_level.name

),

stats_local AS 
(
    SELECT cat.first_level.name, COUNT(distinct id) AS pubs_local, 
    ROUND (
            (COUNT(distinct id) * 100 /   -- calc percentage to total pubs for liliput
                ( 
                    SELECT COUNT(distinct id) 
                    FROM liliput_pubs
                )
            ), 
        2 )  AS pubs_local_pc 
    FROM liliput_pubs, UNNEST(category_for) cat
    GROUP BY cat.first_level.name
)
SELECT stats_local.name, pubs_global, pubs_global_pc, pubs_local, pubs_local_pc, (pubs_global_pc - pubs_local_pc) AS delta
FROM stats_global JOIN stats_local ON stats_global.name = stats_local.name
ORDER BY stats_local.name



name  pubs_global  pubs_global_pc  \
0         Agricultural and Veterinary Sciences      2017344            2.48   
1                          Biological Sciences      8633215           10.60   
2                 Built Environment and Design       455577            0.56   
3                            Chemical Sciences      7572654            9.30   
4   Commerce, Management, Tourism and Services      1699985            2.09   
5                               Earth Sciences      1966038            2.42   
6                                    Economics      1638927            2.01   
7                                    Education      1689933            2.08   
8                                  Engineering     11663754           14.33   
9                       Environmental Sciences      1293430            1.59   
10                     History and Archaeology      2227799            2.74   
11          Information and Computing Sciences      4793802            5.89   
12         Language, Communication and Culture      2365009            2.91   
13                       Law and Legal Studies       826205            1.01   
14                       Mathematical Sciences      4805236            5.90   
15                 Medical and Health Sciences     28785963           35.36   
16            Philosophy and Religious Studies      1591230            1.95   
17                           Physical Sciences      5932268            7.29   
18           Psychology and Cognitive Sciences      3650890            4.48   
19        Studies in Creative Arts and Writing       615514            0.76   
20                    Studies in Human Society      3184273            3.91   
21                                  Technology      1863756            2.29   

    pubs_local  pubs_local_pc  delta  
0          166           0.49   1.99  
1         9041          26.71 -16.11  
2            7           0.02   0.54  
3         7732          22.84 -13.54  
4          613           1.81   0.28  
5           71           0.21   2.21  
6         2241           6.62  -4.61  
7           36           0.11   1.97  
8         3637          10.75   3.58  
9           94           0.28   1.31  
10          42           0.12   2.62  
11         381           1.13   4.76  
12         193           0.57   2.34  
13          76           0.22   0.79  
14        1535           4.54   1.36  
15        6664          19.69  15.67  
16          45           0.13   1.82  
17        8815          26.04 -18.75  
18         430           1.27   3.21  
19         127           0.38   0.38  
20         316           0.93   2.98  
21         795           2.35  -0.06

#### Tip: calc percentage of total

In [19]:
%%bigquery --params $bq_params --project $project_id 

select cat.first_level.name, count(distinct p.id) as pubs_global, 
    ROUND ((count(distinct p.id) * 100 /(SELECT count(*) FROM `dimensions-ai.data_analytics.publications`)), 2 )  as pubs_global_pc 
from `dimensions-ai.data_analytics.publications` p, unnest(category_for) cat
group by cat.first_level.name



pubs_global                                        name  pubs_global_pc
0       1290505                      Environmental Sciences            1.17
1        454265                Built Environment and Design            0.41
2       7562108                           Chemical Sciences            6.85
3      11636113                                 Engineering           10.54
4       3178085                    Studies in Human Society            2.88
5       1696374  Commerce, Management, Tourism and Services            1.54
6       5915637                           Physical Sciences            5.36
7        823865                       Law and Legal Studies            0.75
8        614805        Studies in Creative Arts and Writing            0.56
9       2362047         Language, Communication and Culture            2.14
10      4798149                       Mathematical Sciences            4.35
11      1963135                              Earth Sciences            1.78
12      4781866          Information and Computing Sciences            4.33
13      2224861                     History and Archaeology            2.02
14      2014688        Agricultural and Veterinary Sciences            1.83
15      1860405                                  Technology            1.69
16      8614115                         Biological Sciences            7.81
17      3644099           Psychology and Cognitive Sciences            3.30
18      1634620                                   Economics            1.48
19     28739353                 Medical and Health Sciences           26.04
20      1685678                                   Education            1.53
21      1589440            Philosophy and Religious Studies            1.44

### Funding Sources

Links to grants are determined via the 'grants assignees' relationship (not the publications links). 

In [29]:
%%bigquery --params $bq_params --project $project_id 

WITH liliput AS (
  SELECT researcher_id
  FROM `ds-data-solutions-gbq.liliput01.researchers`
),
funders_all AS
(
SELECT grid.name, funder, count(distinct g.id) AS tot,
  FROM `dimensions-ai.data_analytics.grants` g, unnest(researchers) allres
  JOIN liliput on liliput.researcher_id = allres.id
  JOIN `dimensions-ai.data_analytics.grid` grid ON grid.id = funder
GROUP BY funder, grid.name 
), 
funders_active AS
(
 SELECT funder, count(distinct g.id) AS tot_active,
  FROM `dimensions-ai.data_analytics.grants` g, unnest(researchers) allres
  JOIN liliput on liliput.researcher_id = allres.id
  JOIN `dimensions-ai.data_analytics.grid` grid ON grid.id = funder
  WHERE PARSE_DATE('%Y-%m-%d', end_date) >= CURRENT_DATE()
GROUP BY funder    
)
select funders_all.* , funders_active.tot_active 
from funders_all 
join funders_active on funders_all.funder = funders_active.funder
ORDER BY tot desc

name         funder  tot  \
0          Japan Society for the Promotion of Science  grid.54432.34  304   
1    Directorate for Mathematical & Physical Sciences  grid.457875.c  143   
2      National Institute of General Medical Sciences  grid.280785.0   92   
3                 Directorate for Biological Sciences  grid.457768.f   56   
4                                      Wellcome Trust  grid.52788.30   52   
5                           National Cancer Institute  grid.48336.3a   47   
6   National Institute of Allergy and Infectious D...  grid.419681.3   31   
7                     Deutsche Forschungsgemeinschaft  grid.424150.6   30   
8                         Directorate for Engineering  grid.457810.f   24   
9                      The Research Council of Norway  grid.13985.36   22   
10                          European Research Council  grid.452896.4   18   
11  National Institute of Neurological Disorders a...  grid.416870.c   17   
12            National Heart Lung and Blood Institute  grid.279885.9   17   
13                        National Institute on Aging  grid.419475.a   16   
14                National Institute of Mental Health  grid.416868.5   15   
15                           Medical Research Council  grid.14105.31   15   
16  Engineering and Physical Sciences Research Cou...  grid.421091.f   12   
17             Canadian Institutes of Health Research  grid.248883.d   11   
18  National Institute of Diabetes and Digestive a...  grid.419635.c    8   
19                                 Cancer Research UK  grid.11485.39    7   
20  Directorate for Computer & Information Science...  grid.457785.c    7   
21                    Office of Basic Energy Sciences  grid.452988.a    6   
22        Directorate for Education & Human Resources  grid.457799.1    5   
23  National Institute of Environmental Health Sci...  grid.280664.e    4   
24           American Association For Cancer Research  grid.280840.6    3   
25                   National Institute on Drug Abuse  grid.420090.f    3   
26       National Health and Medical Research Council  grid.431143.0    2   
27                      Swedish Heart-Lung Foundation  grid.453055.2    1   

    tot_active  
0            4  
1            1  
2           13  
3            2  
4           10  
5           13  
6            2  
7            1  
8            1  
9            6  
10           7  
11           4  
12           2  
13           5  
14           2  
15           1  
16           4  
17           1  
18           1  
19           1  
20           1  
21           3  
22           1  
23           2  
24           2  
25           2  
26           1  
27           1